<center><img src="img/Marca-ITBA-Color-ALTA.png" width="250">

<h1>Master en Management & Analytics</h1>
</center>


## Clase 4 - Optimización de hiperparámetros - parte B

#### Referencias y bibliografía de consulta:

- Introduction to Machine Learning with Python by Andreas C. Müller and Sarah Guido (O’Reilly) 2017 
- An Introduction to Statistical Learning with Applications in R by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani (Springer) 2017 
- Python Machine Learning - Second Edition by Sebastian Raschka (Packt) 2017
- Hands-On Machine Learning with Scikit-Learn and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems - Aurélien Géron - 2017
- https://scikit-learn.org/


<img src="img/House sale.jpg" width="400"  >

En la clase de hoy vamos a seguir trabajando con el [dataset](https://geodacenter.github.io/data-and-lab/KingCounty-HouseSales2015/) de precios de venta de viviendas del King County. 

Recordemos cuáles son los atributos del dataset y qué representa cada uno:
Tenemos 21 atributos:
- **id**: identificación
- **date**: fecha de venta       
- **price**: precio de venta. Esta es nuestra `variable objetivo`.
- **bedrooms**: cantidad de habitaciones
- **bathrooms**: cantidad de baños
- **sqft_living**: tamaño de la zona habitable en pies cuadrados
- **sqft_lot**: tamaño del lote en pies cuadrados
- **floors**: cantidad de pisos
- **waterfront**: 1' si la propiedad tiene un frente de agua, 0' si no.
- **view**: un índice de 0 a 4 de lo buena que era la vista de la propiedad
- **condition**: estado de la casa, clasificado del 1 al 5
- **grade**: clasificación según la calidad de la construcción, que se refiere a los tipos de materiales utilizados y a la calidad de la mano de obra. Los edificios de mejor calidad (mayor grado) cuestan más de construir por unidad de medida y tienen mayor valor.
- **sqft_above**: Pies cuadrados sobre el suelo
- **sqft_basement**: Pies cuadrados bajo tierra
- **yr_built**: año de construcción
- **yr_renovated**: año de renovación. 0" si no se ha renovado nunca
- **zipcode**: código postal de 5 dígitos
- **lat**: latitud
- **long**: longitud
- **sqft_living15**: tamaño medio del espacio habitable de las 15 casas más cercanas, en pies cuadrados
- **sqft_lot15**: tamaño medio del lote de las 15 casas más cercanas, en pies cuadrados

Vamos a comenzar, como de costumbre importando algunas de las librerías que vamos a necesitar:

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import warnings
warnings.filterwarnings('ignore')

Creamos un DataFrame tomando el csv con los datos:

In [2]:
dataset = pd.read_csv('data/kc_house_data.csv')

dataset.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


Vamos a realizar las tareas de limpieza que hicimos la clase pasada:

In [3]:
# Casteamos el atributo 'date' a datetime
dataset['date'] = pd.to_datetime(dataset['date'])

# Creamos una variable que represente el año de venta
dataset['year_sold'] = dataset['date'].dt.year

# Corregimos el error de carga en 'bedrooms'
dataset.loc[dataset['bedrooms']==33,'bedrooms'] = 3

Creamos nuestra matriz de features y nuestro vector target.

In [4]:
# Creamos X e y
features_cols = [x for x in dataset.columns if x not in ['price','id', 'date','zipcode']]
X = dataset[features_cols]
y = dataset['price']

# Verificamos el shape y el tipo de X e y:
print("Shape X: {}".format(X.shape))
print("Type X: {}".format(type(X)))
print("Shape y: {}".format(y.shape))
print("Type y: {}".format(type(y)))

Shape X: (21613, 18)
Type X: <class 'pandas.core.frame.DataFrame'>
Shape y: (21613,)
Type y: <class 'pandas.core.series.Series'>


Hacemos el split entre train y test sets:

In [5]:
rs=1

X_train, X_test, y_train, y_test = \
         train_test_split(X, y, test_size=0.3, random_state=rs)

Realizamos la imputación de las variables 'bedrooms' y 'bathrooms' para las casas con valores igual a 0, cuidando de calcular los valores en el set de entrenamiento y utilizando dichos valores para el train y test sets:

In [6]:
X_train['bedrooms'].replace(0, np.nan, inplace=True)
X_train['bathrooms'].replace(0, np.nan, inplace=True)
X_test['bedrooms'].replace(0, np.nan, inplace=True)
X_test['bathrooms'].replace(0, np.nan, inplace=True)

X_train.fillna(X_train.groupby('floors').transform('median'), inplace=True)

dicc_bed = X_train.groupby('floors')['bedrooms'].median().apply(np.ceil).to_dict()
dicc_bath = X_train.groupby('floors')['bathrooms'].median().apply(np.ceil).to_dict()

X_test['bedrooms'] = X_test['bedrooms']\
                            .fillna(X_test['floors'].apply(lambda x: dicc_bed.get(x)))

X_test['bathrooms'] = X_test['bathrooms']\
                            .fillna(X_test['floors'].apply(lambda x: dicc_bath.get(x)))

#### Estandarización

Un aspecto metodológico importante es que para aplicar estas técnicas de regularización, tenemos que normalizar los datos. Esto se debe a que, al penalizar los parámentros por su tamaño, la escala en la que están medidas las variables se vuelve absolutamente relevante.

Con el modelo de regresión lineal sin regularización, el valor de los parámetros compensaba por las unidades de medida de las variables, por lo que no afectaba al resultado del modelo. Al aplicar regularización, no queremos que efectos de escala afecten la imporancia relativa de los parámetros. Por este motivo, estandarizamos a todas las variables, con media $0$ y desvío estándar $1$, de modo tal que de llevar a todas las variables a una escala común. 

Uno de los aspectos que tenemos que saber antes de entrenar un modelo de machine learning es si requiere o no que los datos estén normalizados.

Apliquemos estandarización, transformando las variables para que tengan media 0 $(\mu = 0)$ y desvío estándar 1 $(\sigma = 1)$, aplicando la fórmula:

$$ x' = \frac{x - \mu}{\sigma}$$

In [7]:
from sklearn.preprocessing import StandardScaler
stdscaler = StandardScaler()

X_train_std = pd.DataFrame(stdscaler.fit_transform(X_train), columns=X_train.columns)
X_test_std = pd.DataFrame(stdscaler.transform(X_test), columns=X_test.columns)

Es importante notar que estamos haciendo el `fit_transform()` de `stdscaler` con los datos de entrenamiento. Para los datos de testeo, estamos usando el método `transform`, es decir que normalizamos utilizando las medias y desvíos estándar calculados en el set de testo. Esto es así para evitar **data leakage**, es decir filtración de información del set de testo en el modelo.  


### Support Vector Machines (SVM)

##### SVM es Modelo Discriminante: busca trazar una recta o curva divisoria en el espacio de features.

SVM es un algoritmo que sirve para resolver tanto problemas de clasificación como regresión. 


Pensemos en un problema de clasificación. Queremos dividir las 2 clases. Existen muchas posibles líneas que podríamos trazar para dividir las dos clases ¿Cuál sería la mejor? 

<img src="img/svm1.png" align="center" width="400"/>

Dependiendo la línea que se elija, una nueva observación puede quedar en una u otra clase.

Imaginen que trazamos un “margen” entre la línea discriminante y el punto más cercano de cada clase. Lo que hace el algoritmo SVM es encontrar **la línea discriminante que maximiza el ancho de este “margen”**.

<img src="img/svm2.png" align="center" width="400"/>

Cuando ajustamos el modelo SVM a estos datos obtenemos la línea discriminante que se encuentra en el medio. Las líneas punteadas representan los límites del margen.

Es importante notar que solamente algunos de los puntos tocan los límites del margen. Estos puntos son los “vectores de soporte”. A la hora de hacer el ajuste del modelo, los únicos puntos que importan son estos. 
El comportamiento de los que se encuentran lejos del “decision boundary” no tiene ninguna importancia.

<img src="img/svm3.png" align="center" width="400"/>

##### ¿Qué pasa si no se puede hacer una división perfecta?

Para resolver este problema, el algoritmo incorpora el parámetro C que permite que algunos puntos permanezcan dentro del margen para mejorar el ajuste.

<img src="img/svm4.png" align="center" width="850"/>

##### El hiperparámetro C es uno los que regulariza el trade-off entre sesgo y varianza.

C grande -> menos regularización -> menos tolerancia al error -> márgenes más chicos -> más varianza

C chico -> más regularización -> más tolerancia al error -> márgenes más grandes -> más sesgo


#### Kernel Trick

Sin entrar demasiado en los detalles técnicos, es importante señalar que en SVM, la estimación del modelo implica el producto interno entre todas las observaciones. Recordemos que el producto interno entre 2 vectores nos da una medida de su similitud.

<img src="img/inner_prod.png" align="center" width="350"/>

El SVM lineal puede representarse del siguiente modo:

<img src="img/linear_kernel.png" align="center" width="350"/>

Reemplazando al producto interno con una generalización del tipo:

<img src="img/kernel.png" align="center" width="200"/>

a la que llamamos `kernel`. El `kernel` es una función que cuantifica la similitud entre 2 observaciones. 

Podríamos en cambio utilizar un kernel polinómico:

<img src="img/poly_kernel.png" align="center" width="400"/>

En esencia, se trata de ajustar un clasificador de vectores de soporte en un espacio de mayor dimensión que incluye polinomios de grado *d*, en lugar de hacerlo en el espacio de características original.

Otra alternativa es el kernel radial:

<img src="img/rbf_kernel.png" align="center" width="400"/>

La función kernel en SVM nos dice, dados dos puntos en el espacio de los datos original, cuál es su similitud en el espacio de las features. Permite evitar hacer la transformación de los datos al espacio de las features. Diferentes kernels corresponden a diferentes transformaciones de los datos.

<img src="img/svm5.png" align="center" width="850"/>

Cuanto más chico es gamma, la función de similitud del kernel RBF cae más suavemente con la distancia entre los puntos y por lo tanto genera fronteras de decisión más suaves.

<img src="img/svm6.png" align="center" width="850"/>

En la siguiente imagen podemos observar las fronteras de decisión de un kernel polinómico de grado 3 (izq.) y un kernel RBF para el mismo dataset:

<img src="img/poly_rbf.png" align="center" width="750"/>

### Búsqueda sobre espacios que no son grillas

En algunos casos, probar todas las combinaciones posibles de todos los parámetros, como suele hacer `GridSearchCV`, no es una buena idea. Por ejemplo, SVM tiene un parámetro de `kernel`, y dependiendo del kernel que se elija, otros parámetros serán relevantes. Si kernel='linear', el modelo es lineal, y sólo se utiliza el parámetro C. Si kernel='rbf', se utilizan los parámetros C y gamma. En este caso, la búsqueda de todas las combinaciones posibles de C, gamma y kernel no tendría sentido.

Para tratar este tipo de parámetros "condicionales", `GridSearchCV` permite que `param_grid` sea una lista de diccionarios. Cada diccionario de la lista se expande en una cuadrícula independiente. 

In [8]:
param_grid = [{'kernel': ['rbf'],
               'C': [1000, 10000, 100000, 1000000, 10000000],
               'gamma': [0.001, 0.01, 0.1, 1, 10, 100]},
              {'kernel': ['linear'],
               'C': [1000, 10000, 100000, 1000000, 10000000]},
              {'kernel': ['poly'],
               'C': [1000, 10000, 100000, 1000000, 10000000],
               'degree': [2, 3, 4]}]

print("Lista de grillas:\n{}".format(param_grid))

Lista de grillas:
[{'kernel': ['rbf'], 'C': [1000, 10000, 100000, 1000000, 10000000], 'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}, {'kernel': ['linear'], 'C': [1000, 10000, 100000, 1000000, 10000000]}, {'kernel': ['poly'], 'C': [1000, 10000, 100000, 1000000, 10000000], 'degree': [2, 3, 4]}]


In [9]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(SVR(cache_size=500), param_grid, cv=3, scoring='r2', n_jobs=-1, verbose=2)
grid_search.fit(X_train_std, y_train)
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.5f}".format(grid_search.best_score_))

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best parameters: {'C': 10000000, 'gamma': 0.01, 'kernel': 'rbf'}
Best cross-validation score: 0.81746


In [10]:
y_pred_train_grid = grid_search.predict(X_train_std)
print ('R2 train GridSearchCV: {}'.format(r2_score(y_train,y_pred_train_grid).round(5)))

y_pred_test_grid = grid_search.predict(X_test_std)
print ('R2 test GridSearchCV: {}'.format(r2_score(y_test,y_pred_test_grid).round(5)))

R2 train GridSearchCV: 0.86332
R2 test GridSearchCV: 0.82399
